In [1]:
import pyaudio
import librosa
import tensorflow as tf
import os
os.environ['KERAS_BACKEND']='tensorflow'
import keras
from keras.utils import np_utils
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

In [14]:
#创建自定义训练数据列表
#audio_path为音频文件路径，用户需要提前把音频数据集存放在my_dataset/audio目录下，每个文件夹存放一个类别的音频数据，如my_dataset/audio/鸟叫声/...
#生成的数据类别的格式为音频路径\t音频对应的类别标签
def get_data_list(audio_path, list_path):
    sound_sum = 0
    audios = os.listdir(audio_path)

    f_train = open(os.path.join(list_path, 'train_list.txt'), 'w')
    f_test = open(os.path.join(list_path, 'test_list.txt'), 'w')

    for i in range(len(audios)):
        sounds = os.listdir(os.path.join(audio_path, audios[i]))
        for sound in sounds:
            sound_path = os.path.join(audio_path, audios[i], sound)
            t = librosa.get_duration(filename=sound_path)
            # [可能需要修改参数] 过滤小于2.1秒的音频
            if t >= 2:
                if sound_sum % 5 == 0:
                    f_test.write('%s\t%d\n' % (sound_path, i))
                else:
                    f_train.write('%s\t%d\n' % (sound_path, i))
                sound_sum += 1
        print("Audio：%d/%d" % (i + 1, len(audios)))

    f_test.close()
    f_train.close()

if __name__ == '__main__':
    get_data_list('my_dataset/audio', 'my_dataset')

Audio：1/2
Audio：2/2


In [3]:
# 创建UrbanSound8K数据列表
def get_urbansound8k_list(path, urbansound8k_cvs_path):
    data_list = []
    data = pd.read_csv(urbansound8k_cvs_path)
    # 过滤掉长度少于3秒的音频
    valid_data = data[['slice_file_name', 'fold', 'classID', 'class']][data['end'] - data['start'] >= 3]
    valid_data['path'] = 'fold' + valid_data['fold'].astype('str') + '/' + valid_data['slice_file_name'].astype('str')
    for row in valid_data.itertuples():
        data_list.append([row.path, row.classID])

    f_train = open(os.path.join(path, 'train_list.txt'), 'w')
    f_test = open(os.path.join(path, 'test_list.txt'), 'w')

    for i, data in enumerate(data_list):
        sound_path = os.path.join('dataset/UrbanSound8K/audio/', data[0])
        if i % 100 == 0:
            f_test.write('%s\t%d\n' % (sound_path, data[1]))
        else:
            f_train.write('%s\t%d\n' % (sound_path, data[1]))

    f_test.close()
    f_train.close()


if __name__ == '__main__':
    get_urbansound8k_list('dataset', 'dataset/UrbanSound8K/metadata/UrbanSound8K.csv')

In [16]:
# 获取浮点数组
def _float_feature(value):
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


# 获取整型数据
def _int64_feature(value):
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


# 把数据添加到TFRecord中
def data_example(data, label):
    feature = {
        'data': _float_feature(data),
        'label': _int64_feature(label),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))


# 开始创建tfrecord数据
def create_data_tfrecord(data_list_path, save_path):
    with open(data_list_path, 'r') as f:
        data = f.readlines()
    with tf.io.TFRecordWriter(save_path) as writer:
        for d in tqdm(data):
            try:
                path, label = d.replace('\n', '').split('\t')
                wav, sr = librosa.load(path, sr=48000)
                wav_output = []
                '''
                intervals = librosa.effects.split(wav, top_db=20) #返回有声音的部分，即裁剪静音部分
                for sliced in intervals:
                    wav_output.extend(wav[sliced[0]:sliced[1]])
                '''
                wav_output.extend(wav)
                # [可能需要修改参数] 音频长度 48000 * 秒数
                wav_len = int(48000 * 2)
                for i in range(3):
                    # 裁剪过长的音频，过短的补0
                    if len(wav_output) > wav_len:
                        l = len(wav_output) - wav_len
                        r = random.randint(0, l)
                        wav_output = wav_output[r:wav_len + r]
                    else:
                        wav_output.extend(np.zeros(shape=[wav_len - len(wav_output)], dtype=np.float32))
                    #wav_output = np.array(wav_output)
                    # 转成梅尔频谱
                    #data = librosa.feature.melspectrogram(y=np.array(wav_output), sr=sr, n_fft=512, hop_length=256).reshape(-1).tolist()
                    
                    #转化为梅尔倒谱系数mfcc
                    melspec = librosa.feature.melspectrogram(y=np.array(wav_output), sr=sr, n_fft=512, hop_length=256)
                    logmelspec = librosa.power_to_db(melspec)
                    data = librosa.feature.mfcc(S=logmelspec, n_mfcc=40).reshape(-1).tolist()
                    
                    if len(data) != 40 * 376: continue
                    tf_example = data_example(data, int(label))
                    writer.write(tf_example.SerializeToString())
                    if len(wav_output) <= wav_len:
                        break
            except Exception as e:
                print(e)


if __name__ == '__main__':
    create_data_tfrecord('my_dataset/train_list.txt', 'my_dataset/new_mfccs_train.tfrecord')
    create_data_tfrecord('my_dataset/test_list.txt', 'my_dataset/new_mfccs_test.tfrecord')

 57%|████████████████████████████████████████████▌                                 | 1795/3141 [01:40<01:16, 17.62it/s]C:\Users\xuqiu\anaconda3\lib\site-packages\librosa\filters.py:238: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████| 786/786 [01:31<00:00,  8.56it/s]


In [175]:
def create_dataset(data_list_path):
    X=[]
    Y=[]
    with open(data_list_path, 'r') as f:
        data = f.readlines()
    for d in tqdm(data):
        path, label = d.replace('\n', '').split('\t')
        wav, sr = librosa.load(path, sr=48000)
        wav_output = []
        wav_output.extend(wav)
        # [可能需要修改参数] 音频长度 48000 * 秒数
        wav_len = int(48000 * 2)
        for i in range(5):
            # 裁剪过长的音频，过短的补0
            if len(wav_output) > wav_len:
                l = len(wav_output) - wav_len
                r = random.randint(0, l)
                wav_output = wav_output[r:wav_len + r]
            else:
                wav_output.extend(np.zeros(shape=[wav_len - len(wav_output)], dtype=np.float32))
            #wav_output = np.array(wav_output)
            # 转成梅尔频谱
            #data = librosa.feature.melspectrogram(y=np.array(wav_output), sr=sr, n_fft=512, hop_length=256)
                    
            #转化为梅尔倒谱系数mfcc
            melspec = librosa.feature.melspectrogram(y=np.array(wav_output), sr=sr, n_fft=512, hop_length=256)
            logmelspec = librosa.power_to_db(melspec)
            data = librosa.feature.mfcc(S=logmelspec, n_mfcc=40).reshape(40,376,1)
                    
            if data.shape != (40,376,1): continue
            X.append(data)
            Y.append(int(label))
    X_dataset, Y_dataset = shuffle(np.array(X), np.array(Y))  #混洗数据
    return X_dataset, Y_dataset

if __name__ == '__main__':
    #X_train, Y_train = create_dataset('my_dataset/train_list.txt')
    X_test, Y_test = create_dataset('my_dataset/test_list.txt')

100%|██████████| 453/453 [00:58<00:00,  7.81it/s]


In [176]:
Y_test[:20]

array([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0])

In [123]:
X_train.shape

(9055, 40, 376, 1)

In [183]:
Y_test_onehot_new =np_utils.to_categorical(Y_test, num_classes=3)
Y_test_onehot_new[:20]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)

In [177]:
#lb = LabelEncoder()
Y_test_onehot =np_utils.to_categorical(LabelEncoder().fit_transform(Y_test))
Y_test_onehot[:20]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [29]:
Y_train.shape

(9055, 2)

In [25]:
#读取TFrecord数据
import tensorflow as tf

def _parse_data_function(example):
    # [可能需要修改参数】 设置的梅尔频谱的shape相乘的值
    data_feature_description = {
        'data': tf.io.FixedLenFeature([15040], tf.float32),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    return tf.io.parse_single_example(example, data_feature_description)


def train_reader_tfrecord(data_path, num_epochs, batch_size):
    raw_dataset = tf.data.TFRecordDataset(data_path)
    train_dataset = raw_dataset.map(_parse_data_function)
    train_dataset = train_dataset.shuffle(buffer_size=1000) \
        .repeat(count=num_epochs) \
        .batch(batch_size=batch_size) \
        .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return train_dataset


def test_reader_tfrecord(data_path, batch_size):
    raw_dataset = tf.data.TFRecordDataset(data_path)
    test_dataset = raw_dataset.map(_parse_data_function)
    test_dataset = test_dataset.shuffle(buffer_size=1000)
    test_dataset = test_dataset.batch(batch_size=batch_size)
    return test_dataset

In [27]:
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten
from keras.models import Sequential #models.Sequential，用来一层一层一层的去建立神经层
from keras.layers import Dense, Activation #layers.Dense 意思是这个神经层是全连接层 layers.Activation 激励函数
from keras.optimizers import Adam

#建立卷积神经网络CNN模型
class_dim = 2 #总共两种声音，无人机与非无人机
EPOCHS = 10
BATCH_SIZE=32

#添加第一个卷积层，滤波器数量为32，大小是5*5，Padding方法是same即不改变数据的长度和宽带。
model = Sequential()
model.add(Convolution2D(
    filters=32,
    kernel_size=5,
    strides=1,
    padding='same',      # Padding method
    data_format='channels_last',
    input_shape=(40, 376, 1),
))
model.add(Activation('relu'))

#第一层 pooling（池化，下采样），分辨率长宽各降低一半，输出数据shape为（20，188, 32）
model.add(MaxPooling2D(
    pool_size=2,
    strides=2,
    padding='same',    # Padding method
    data_format='channels_last',
))

#再添加第二卷积层，输出滤波器数量64，卷积核大小5
model.add(Convolution2D(64, 5, strides=1, padding='same', data_format='channels_last'))
model.add(Activation('relu'))

#第二层池化层
model.add(MaxPooling2D(2, 2, 'same', data_format='channels_last'))

#经过以上处理后，数据shape为（10，94，64），需展开为一维，再连接全连接层
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))

#添加全连接层（输出层）
model.add(Dense(class_dim))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 40, 376, 32)       832       
_________________________________________________________________
activation_8 (Activation)    (None, 40, 376, 32)       0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 20, 188, 32)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 20, 188, 64)       51264     
_________________________________________________________________
activation_9 (Activation)    (None, 20, 188, 64)       0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 10, 94, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 60160)            

In [28]:
#编译，设置adam优化方法
adam = Adam(lr=1e-3)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])

In [27]:
model.fit(X_train, Y_train, epochs=10, batch_size=32)

Epoch 1/10


ValueError: in user code:

    C:\Users\xuqiu\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\xuqiu\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\xuqiu\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\xuqiu\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\xuqiu\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\xuqiu\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:788 run_step  **
 

In [40]:
loss, accuracy = model.evaluate(X_test, Y_test)

71/71 [==============================] - 8s 111ms/step - loss: 0.0032 - accuracy: 0.3863


In [48]:
#建立模型
class_dim = 10 #总共十种声音
EPOCHS = 1
BATCH_SIZE=32

model = tf.keras.models.Sequential([
    tf.keras.applications.ResNet50V2(include_top=False, weights=None, input_shape=(128, 128, 1)), #默认为channel last，channel填在最后
    tf.keras.layers.ActivityRegularization(l2=0.5), #对基于代价函数的输入活动应用一个正则化更新
    tf.keras.layers.Dropout(rate=0.5),   #防止过拟合
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(units=class_dim, activation=tf.nn.softmax)
])

model.summary() #显示模型的结构


# 定义优化方法
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

train_dataset = train_reader_tfrecord('dataset/train.tfrecord', EPOCHS, batch_size=BATCH_SIZE)
test_dataset = test_reader_tfrecord('dataset/test.tfrecord', batch_size=BATCH_SIZE)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 4, 4, 2048)        23558528  
_________________________________________________________________
activity_regularization_6 (A (None, 4, 4, 2048)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 4, 4, 2048)        0         
_________________________________________________________________
global_max_pooling2d_6 (Glob (None, 2048)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                20490     
Total params: 23,579,018
Trainable params: 23,533,578
Non-trainable params: 45,440
_________________________________________________________________


In [20]:
train_dataset = train_reader_tfrecord('my_dataset/new_mfccs_train.tfrecord', EPOCHS, batch_size=BATCH_SIZE)
test_dataset = test_reader_tfrecord('my_dataset/new_mfccs_test.tfrecord', batch_size=BATCH_SIZE)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

In [187]:
#训练模型
for batch_id, data in enumerate(train_dataset):
    # [可能需要修改参数】 设置的梅尔频谱的shape
    #要注意的是在创建TFRecord文件时，已经把音频数据的梅尔频谱转换为一维list了，所以在数据输入到模型前，需要把数据reshape为之前的shape，操作方式为reshape((-1, 128, 128, 1))
    sounds = data['data'].numpy().reshape((-1, 40, 376, 1))
    print(sounds.shape)
    labels = data['label'].numpy()
    labels =np_utils.to_categorical(labels, num_classes=2)
    # 执行训练
    with tf.GradientTape() as tape:
        predictions = model(sounds)
        # 获取损失值
        #train_loss = tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)
        train_loss = tf.keras.losses.binary_crossentropy(labels, predictions)
        train_loss = tf.reduce_mean(train_loss)
        # 获取准确率
        #train_accuracy = tf.keras.metrics.sparse_categorical_accuracy(labels, predictions)
        train_accuracy = tf.keras.metrics.binary_accuracy(labels, predictions)
        train_accuracy = np.sum(train_accuracy.numpy()) / len(train_accuracy.numpy())

    # 更新梯度
    gradients = tape.gradient(train_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    if batch_id % 20 == 0:
        print("Batch %d, Loss %f, Accuracy %f" % (batch_id, train_loss.numpy(), train_accuracy))

    if batch_id % 200 == 0 and batch_id != 0:
        test_losses = list()
        test_accuracies = list()
        for d in test_dataset:
            # [可能需要修改参数】 设置的梅尔频谱的shape
            test_sounds = d['data'].numpy().reshape((-1, 40, 376, 1))
            test_labels = d['label'].numpy()
            test_labels =np_utils.to_categorical(test_labels, num_classes=2)

            test_result = model(test_sounds)
            # 获取损失值
            #test_loss = tf.keras.losses.sparse_categorical_crossentropy(test_labels, test_result)
            test_loss = tf.keras.losses.binary_crossentropy(test_labels, test_result)
            test_loss = tf.reduce_mean(test_loss)
            test_losses.append(test_loss)
            # 获取准确率
            #test_accuracy = tf.keras.metrics.sparse_categorical_accuracy(test_labels, test_result)
            test_accuracy = tf.keras.metrics.binary_accuracy(test_labels, test_result)
            test_accuracy = np.sum(test_accuracy.numpy()) / len(test_accuracy.numpy())
            test_accuracies.append(test_accuracy)

        print('=================================================')
        print("Test, Loss %f, Accuracy %f" % (
            sum(test_losses) / len(test_losses), sum(test_accuracies) / len(test_accuracies)))
        print('=================================================')

        # 保存模型
        model.save(filepath='models/test.h5')

(32, 40, 376, 1)
Batch 0, Loss 6.277274, Accuracy 0.000000
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
Batch 20, Loss 0.692296, Accuracy 0.656250
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
Batch 40, Loss 0.698357, Accuracy 0.375000
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 

(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
Batch 420, Loss 0.000002, Accuracy 1.000000
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
Batch 440, Loss 0.000000, Accuracy 1.000000
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
(32, 40, 376, 1)
Batch 460, Loss 0.000002, Accuracy 1.000000
(32, 40, 376, 1)
(32, 40, 376, 1)
(

In [95]:
train_dataset = train_reader_tfrecord('my_dataset/train.tfrecord', EPOCHS, batch_size=BATCH_SIZE)

In [130]:
train_dataset

AttributeError: 'PrefetchDataset' object has no attribute 'data'

In [135]:
test_dataset = test_reader_tfrecord('my_dataset/test.tfrecord', batch_size=BATCH_SIZE)

In [136]:
test_dataset

<MapDataset shapes: {data: (15040,), label: ()}, types: {data: tf.float32, label: tf.int64}>

In [29]:
feature_description = { # 定义Feature结构，告诉解码器每个Feature的类型是什么
    'data': tf.io.FixedLenFeature([15040], tf.float32),
    #'data': tf.io.FixedLenFeature([48128], tf.float32),
    'label': tf.io.FixedLenFeature([], tf.int64),
}

def read_and_decode(example):
    feature_dict = tf.io.parse_single_example(example, feature_description)
    data = feature_dict['data']
    data = tf.reshape(data,[40, 376, 1])
    #data = tf.reshape(data,[128, 376, 1])
    label = feature_dict['label']
    label = tf.one_hot(label,2)
    return data, label

In [30]:
tfrecord_file = 'my_dataset/new_mfccs_train.tfrecord'
dataset = tf.data.TFRecordDataset(tfrecord_file)
'''
dataset = dataset.map(read_and_decode)
dataset = dataset.shuffle(buffer_size = 100)
dataset = dataset.repeat(count=EPOCHS)
dataset = dataset.batch(batch_size = BATCH_SIZE)
'''
dataset = dataset.map(read_and_decode)
dataset = dataset.shuffle(buffer_size=1000) \
        .repeat(count=EPOCHS) \
        .batch(batch_size=BATCH_SIZE) \
        .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
model.fit(dataset)

982/982 [==============================] - 541s 551ms/step - loss: 20.5494 - binary_accuracy: 0.9654


In [44]:
model = tf.keras.models.load_model('models/test.h5')
model

In [31]:
tfrecord_file = 'my_dataset/new_mfccs_test.tfrecord'
dataset = tf.data.TFRecordDataset(tfrecord_file) 

dataset = dataset.map(read_and_decode)
dataset = dataset.shuffle(buffer_size = 100)
dataset = dataset.batch(batch_size = 32)

loss, accuracy = model.evaluate(dataset)

25/25 [==============================] - 3s 101ms/step - loss: 0.0037 - binary_accuracy: 0.9987


In [41]:
model.save(filepath='models/test.h5')

In [153]:
dataset = tf.data.Dataset.from_tensor_slices((X,Y)) 

NameError: name 'X' is not defined

In [146]:
def read_and_decode(filename_queue):     #输入文件名队列
    #reader=tf.TFRecordReader()
    #_,serialized_example=reader.read(filename_queue)
    serialized_example=tf.data.TFRecordDataset(tfrecord_file)
    #解析一个example,如果需要解析多个样例，使用parse_example函数
    features=tf.io.parse_single_example(   
            serialized_example,
            #必须写明feature里面的key的名称
            features={
                     'data': tf.io.FixedLenFeature([15040], tf.float32),
                     'label': tf.io.FixedLenFeature([], tf.int64),
                      })
    #对于BytesList,要重新进行编码，把string类型的0维Tensor变成uint8类型的一维Tensor
    #image = tf.decode_raw(features['image_raw'], tf.uint8)
    #image.set_shape([IMAGE_PIXELS])
    #tensor("input/DecodeRaw:0",shape=(784,),dtype=uint8)

    #image张量的形状为：tensor("input/sub:0",shape=(784,),dtype=float32)
    #image = tf.cast(image, tf.float32) * (1. / 255) - 0.5

    #把标记从uint8类型转换为int32类性
    #label张量的形状为tensor（“input/cast_1:0",shape=(),dtype=int32)
    #label = tf.cast(features['label'], tf.int32)
    data = features['data']
    data = tf.reshape(data,(40, 376, 1))
    label = features['label']
    return data,label

In [147]:
data,label=read_and_decode('my_dataset/train.tfrecord')

ValueError: Attempt to convert a value (<TFRecordDatasetV2 shapes: (), types: tf.string>) with an unsupported type (<class 'tensorflow.python.data.ops.readers.TFRecordDatasetV2'>) to a Tensor.

In [192]:
model(sounds)

<tf.Tensor: shape=(30, 2), dtype=float32, numpy=
array([[8.7199126e-34, 1.0000000e+00],
       [2.4190417e-11, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.2443055e-09, 1.0000000e+00],
       [2.9277664e-11, 1.0000000e+00],
       [8.4347731e-14, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [5.8341115e-14, 1.0000000e+00],
       [1.0000000e+00, 4.7905309e-18],
       [3.9787279e-23, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.2653178e-33, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [9.0344907e-23, 1.0000000e+00],
       [3.8353136e-12, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [2.9766440e-13, 1.0000000e+00],
       [7.2514921e-18, 1.0000000e+00],
       [1.1259031e-19, 1.0000000e+00],
       [1.6270456e-17, 1.0000000e+00],
       [1.4857785e-15, 1.0000000e+00],
       [2.1330684e-21, 1.0000000e+00],
       [1.00000

In [28]:
result = model.predict(sounds)
lab = tf.argmax(result, 1)
lab

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0], dtype=int64)>

In [20]:
type(sounds)

numpy.ndarray

In [40]:
sounds.shape

(1, 128, 128, 1)

In [41]:
sounds[0].shape

(128, 128, 1)

In [14]:
type(labels)

tensorflow.python.framework.ops.EagerTensor

In [15]:
labels.shape

TensorShape([32])

In [29]:
labels

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0], dtype=int64)>

In [16]:
tf.one_hot(labels,2)

<tf.Tensor: shape=(32, 2), dtype=float32, numpy=
array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)>

In [42]:
labels.numpy()

array([1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1], dtype=int64)

In [114]:
labels.numpy().shape

(22,)

In [115]:
print(labels.numpy())

[1 0 1 1 1 0 0 1 1 0 0 1 1 1 1 0 0 0 0 0 1 1]


In [7]:
def abs_subtract(array, num): #绝对值减:正的减，负的加
    coefficient=(np.ceil(array/(np.abs(array)*1.1).max())-0.5)*2
    return array-coefficient*num

def eliminate_dc(signal): #消除直流分量
    dc = np.mean(np.abs(signal))
    return abs_subtract(signal,dc)

In [33]:
#预测

# 读取音频数据
def load_data(data_path):
    wav, sr = librosa.load(data_path, sr=48000)
    '''
    intervals = librosa.effects.split(wav, top_db=20) #返回有声音的部分，即裁剪静音部分
    for sliced in intervals:
        wav_output.extend(wav[sliced[0]:sliced[1]])
    '''
    wav_output = []
    wav_output.extend(wav)
    #assert len(wav_output) >= 8000, "有效音频小于0.5s"
    wav_len = int(48000 * 2)
    if len(wav_output) > wav_len:
        l = len(wav_output) - wav_len
        r = random.randint(0, l)
        wav_output = wav_output[r:wav_len + r]
    else:
        wav_output.extend(np.zeros(shape=[wav_len - len(wav_output)], dtype=np.float32))
    melspec = librosa.feature.melspectrogram(y=np.array(wav_output), sr=sr, n_fft=512, hop_length=256)
    #melspec = melspec[np.newaxis, ..., np.newaxis]
    logmelspec = librosa.power_to_db(melspec)
    data = librosa.feature.mfcc(S=logmelspec, n_mfcc=40)
    data = data[np.newaxis, ..., np.newaxis]  #reshape为神经网络的四维输入格式，相当于reshape((1, 40, 376, 1))
    
    #return melspec
    return data


def infer(audio_path):
    data = load_data(audio_path)
    result = model.predict(data)
    lab = tf.argmax(result, 1)
    return lab

'''
if __name__ == '__main__':
    # 要预测的音频文件
    path = 'save_audio/无人机室内8/0f84731c-97ab-11eb-bef9-80fa5b45a5aa.wav'
    label = infer(path)
    print('音频：%s 的预测结果标签为：%d' % (path, label))
'''

"\nif __name__ == '__main__':\n    # 要预测的音频文件\n    path = 'save_audio/无人机室内8/0f84731c-97ab-11eb-bef9-80fa5b45a5aa.wav'\n    label = infer(path)\n    print('音频：%s 的预测结果标签为：%d' % (path, label))\n"

In [9]:
int(label)

0

In [6]:
type(label)

tensorflow.python.framework.ops.EagerTensor

In [26]:
sounds = os.listdir('save_audio/无人机室内1')
len(sounds)

166

In [51]:
#检测率
path='save_audio/无人机室内7'
sounds = os.listdir(path)
correct_num=0
wrong=[]
for sound in tqdm(sounds):
    sound_path = os.path.join(path, sound)
    #wav, sr = librosa.load(path, sr=48000)
    label = infer(sound_path)
    print('音频：%s 的预测结果标签为：%d' % (sound_path, label))
    if label==0:
        correct_num+=1
    else:
        wrong.append(sound)
print('样本总数为%d 正确检测的个数为%d 检测率为%f' % (len(sounds), correct_num, correct_num/len(sounds)))
print(wrong)

  1%|▌                                                                                 | 2/318 [00:00<00:26, 11.98it/s]

音频：save_audio/无人机室内7\94b8629c-9ccc-11eb-84b0-80fa5b45a5aa.wav 的预测结果标签为：0
音频：save_audio/无人机室内7\94b8d77f-9ccc-11eb-bc2b-80fa5b45a5aa.wav 的预测结果标签为：0
音频：save_audio/无人机室内7\94b8ff6a-9ccc-11eb-8f78-80fa5b45a5aa.wav 的预测结果标签为：0


  2%|█▌                                                                                | 6/318 [00:00<00:25, 12.26it/s]

音频：save_audio/无人机室内7\94b9257e-9ccc-11eb-a94d-80fa5b45a5aa.wav 的预测结果标签为：0
音频：save_audio/无人机室内7\94ba0f34-9ccc-11eb-90f5-80fa5b45a5aa.wav 的预测结果标签为：0
音频：save_audio/无人机室内7\94ba3731-9ccc-11eb-a4f9-80fa5b45a5aa.wav 的预测结果标签为：0


  3%|██                                                                                | 8/318 [00:00<00:24, 12.42it/s]

...

音频：save_audio/无人机室内7\94e60b76-9ccc-11eb-8c10-80fa5b45a5aa.wav 的预测结果标签为：0
音频：save_audio/无人机室内7\94e620aa-9ccc-11eb-ada3-80fa5b45a5aa.wav 的预测结果标签为：0
音频：save_audio/无人机室内7\94e66631-9ccc-11eb-9dab-80fa5b45a5aa.wav 的预测结果标签为：0


 99%|██████████████████████████████████████████████████████████████████████████████▉ | 314/318 [00:26<00:00, 11.80it/s]

音频：save_audio/无人机室内7\94e6a5fa-9ccc-11eb-93e2-80fa5b45a5aa.wav 的预测结果标签为：0
音频：save_audio/无人机室内7\94e6ce12-9ccc-11eb-83d8-80fa5b45a5aa.wav 的预测结果标签为：0
音频：save_audio/无人机室内7\94e71d83-9ccc-11eb-89ad-80fa5b45a5aa.wav 的预测结果标签为：0


100%|████████████████████████████████████████████████████████████████████████████████| 318/318 [00:26<00:00, 11.80it/s]

音频：save_audio/无人机室内7\94e741cd-9ccc-11eb-a8c4-80fa5b45a5aa.wav 的预测结果标签为：0
音频：save_audio/无人机室内7\94e768bd-9ccc-11eb-8212-80fa5b45a5aa.wav 的预测结果标签为：0
音频：save_audio/无人机室内7\94e78fb8-9ccc-11eb-8a1a-80fa5b45a5aa.wav 的预测结果标签为：0
样本总数为318 正确检测的个数为318 检测率为1.000000
[]


In [10]:
#虚警率
sounds = os.listdir('dataset/UrbanSound8K/audio/fold2')
wrong_num=0
for sound in tqdm(sounds):
    if sound[-4:] != '.wav':
        continue
    sound_path = os.path.join('dataset/UrbanSound8K/audio/fold2', sound)
    #wav, sr = librosa.load(path, sr=48000)
    label = infer(sound_path)
    print('音频：%s 的预测结果标签为：%d' % (sound_path, label))
    if label==0:
        wrong_num+=1
print('虚报的个数为%d 虚警率为%f' % (wrong_num, wrong_num/len(sounds)))

  0%|▏                                                                                 | 2/889 [00:00<01:37,  9.09it/s]

音频：dataset/UrbanSound8K/audio/fold2\100652-3-0-0.wav 的预测结果标签为：1


  0%|▎                                                                                 | 3/889 [00:00<02:21,  6.28it/s]

音频：dataset/UrbanSound8K/audio/fold2\100652-3-0-1.wav 的预测结果标签为：1


  0%|▎                                                                                 | 4/889 [00:00<02:52,  5.12it/s]

...

音频：dataset/UrbanSound8K/audio/fold2\98202-9-0-4.wav 的预测结果标签为：1
音频：dataset/UrbanSound8K/audio/fold2\98202-9-0-7.wav 的预测结果标签为：1


100%|████████████████████████████████████████████████████████████████████████████████| 889/889 [03:11<00:00,  7.40it/s]

音频：dataset/UrbanSound8K/audio/fold2\98202-9-1-20.wav 的预测结果标签为：1
音频：dataset/UrbanSound8K/audio/fold2\98202-9-1-27.wav 的预测结果标签为：1
音频：dataset/UrbanSound8K/audio/fold2\98202-9-1-5.wav 的预测结果标签为：1


100%|████████████████████████████████████████████████████████████████████████████████| 889/889 [03:11<00:00,  4.64it/s]

虚报的个数为24 虚警率为0.026997
